# Sniper Ability 

Granular PPT data presents the opportunity to calculate a precise location for which the shooter is targetting the goal frame. In the past, we could roughly see where the shooter is aiming on the net, but there was never the capability to calculate precise coordinates. The difference between a save and a goal can be a matter of inches. In this script, we use the precise coordinates for which the shooter is aiming at the goal frame, and calculate the distance to the nearest post/bar. This will give a measure of each players "sniper ability"/ their ability to "pick" the corners. This could be used as a feature of an Expected Goals (xG) model. 

In [34]:
"""
Shots
Uses the shot records with the puck tracking adjacent to each shot record from the ShotEvents table
to calculate the coordinates of the puck when there is a shot and appends the calculation back 
to the original record.
Usage:
    sh init.sh
"""

# Module imports
import logging
import json
import os
import decimal
import time
import boto3

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import numpy as np
from pandas import json_normalize
from decimal import Decimal
from datetime import timedelta, datetime
from scipy.signal import butter,filtfilt


from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
# from Producer_Helpers import DecimalEncoder

logger = logging
logger.basicConfig(format="%(asctime)s :: %(levelname)-5s :: %(message)s",
    level=os.environ.get('LOGGING_LEVEL'))

# AWS Resources
DYNAMODB = boto3.resource('dynamodb')
EVENT_TABLE = DYNAMODB.Table(os.environ.get('EVENT_TABLE', 'Event'))
SHOT_EVENTS_TABLE = DYNAMODB.Table(os.environ.get('SHOT_EVENTS_TABLE', 'ShotEvents'))

tomorrow = datetime.now() + timedelta(0.5)
TTL = tomorrow.timestamp()


def get_event():
    """Scan the Event table for events that have a timestamp
    within 12 hrs to now. Keep trying until call returns an event_id
    Return the event_id.
    """

    while True:

        try:
            logging.info("Getting the event_id from the Event table")
            now = int(time.time())
            response = EVENT_TABLE.scan(
                FilterExpression=Key('TimeStamp').between(
                    Decimal(now - 25000), Decimal(now)),
                ProjectionExpression="EventId",
            )
            event_id = (json.dumps(
                response['Items'][0]['EventId'])).strip('\"')

        except ClientError as e:
            logging.error("Error: EventId not found. {}".format(
                          e.response['Error']['Message']))
            logging.error("Retrying...")
            continue

        except IndexError as e:
            logging.error("Error: EventId not found. {}".format(str(e)))
                        #   e.response['Error']['Message'])
            logging.error("Retrying...")
            continue
        break

    return event_id

def get_shot_events(event_id = 'HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125'):
    """Query the Shot Events table to get the shot events for the given event_id.
    Return shot events as dataframe.
    """

    logger.info("Scanning Shot Events Table for shots...")

    try:
        response = SHOT_EVENTS_TABLE.scan(
            FilterExpression=Attr("EventId").eq(event_id),
            ProjectionExpression="MinorType, ShotUTC, PuckTracking, EndUTC",
        )
        shot_list = response["Items"]

        # Inspect if more pages available in the table
        while "LastEvaluatedKey" in response:
            response = SHOT_EVENTS_TABLE.scan(
                FilterExpression=Attr("EventId").eq(event_id),
                ProjectionExpression="MinorType, ShotUTC, PuckTracking, EndUTC",
                ExclusiveStartKey=response["LastEvaluatedKey"],
            )
            shot_list.extend(response["Items"])

    except ClientError as e:
        logger.info("Error: No data yet.", e.response["Error"]["Message"])

    except IndexError as e:
        logger.info("Error: No data yet.", e.response["Error"]["Message"])

    if len(shot_list) > 0:
        shot_event_df = json_normalize(shot_list)
        logger.info("Shots found.")
        return shot_event_df
    else:
        logger.info("No shot events created yet.")
        return pd.DataFrame() 


def butter_lowpass_filter(data):

    """
    This function applies a butterworth low-pass filter across an array.
    While a butterworth filter is traditionally used for cyclical noise, it works well in this application.


    :param data:    array, the data you would like filtered
    :return:        array, the filtered data
    """
    
    T = 4         # Sample Period
    fs = 60       # sample rate, Hz
    cutoff = 8    # desired cutoff frequency of the filter, Hz
    nyq = 0.5 * fs  # Nyquist Frequency
    order = 2       # sin wave can be approx represented as quadratic
    n = int(T * fs) # total number of samples
    
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # Apply the filter in both directions. This is how lag in filtering is avoided
    # Pad data with zeros before and after input signal
    padlen = 5 * max(len(a), len(b))
    data_padded = np.pad(data, (padlen,padlen), mode='constant', constant_values=0)
    y = filtfilt(b, a, data_padded)
    # remove padding from output signal
    y = y[padlen:-padlen]
    return y

def find_nearest(array, value):

    """
    This function takes an array and a supplied value, and finds the index of the closest value to the supplied value in the array
   
    :param array:   array, contains values
    :param value:   number, value from which you want to find the index to the nearest value
    :return:        int, index within the array nearest to the supplied value
    """

    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def clean_string(string):
    # Remove anything that isn't a number or decimal
    cleaned_string = ''.join(filter(lambda char: char.isdigit() or char == '.', string))
    # Strip any extra decimal points or leading/trailing whitespace
    cleaned_string = cleaned_string.strip().strip('.')
    # Convert to decimal
    try:
        decimal_value = Decimal(cleaned_string)
    except:
        decimal_value = None
    return decimal_value

def shot_result_coords_new(shot_event_df):
    # globals
    NET_X_POSITION = 89

    # remove + replace quotations 
    # shot_event_df = shot_event_df.replace('""','"',regex=True)
    shot_event_df = shot_event_df.reset_index()

    # empty list for new col where updated calcs will be store for each row of df
    shot_outcome_coords = []

    # puck tracking
    for i in range(len(shot_event_df)):
        # check if rows haven't been updated
        if i+1 > len(shot_outcome_coords): 
            index = i
            col = "PuckTracking"
            cell_val = shot_event_df.iloc[index][col]
            puck_data = json_normalize(json.loads(cell_val))
        
            # start and end UTC's from record
            shot_start_time = shot_event_df.iloc[i]['ShotUTC']
            shot_start_time = float(shot_start_time)
            shot_end_time = shot_event_df.iloc[i]['EndUTC']
            start_index = find_nearest(puck_data['LocationUTC'], shot_start_time)
        
            # fill na for calculations
            puck_data[['Velocity.X', 'Velocity.Y', 'Velocity.Z', 'Location.Z']] = puck_data[['Velocity.X', 'Velocity.Y', 'Velocity.Z', 'Location.Z']].fillna(0)
            
            # FEATURE ENGINEERING
            velo_x = np.array(puck_data['Velocity.X'])
            velo_y = np.array(puck_data['Velocity.Y'])    
            velo_z = np.array(puck_data['Velocity.Z'])
            loc_x = np.array(puck_data['Location.X'])
            loc_y = np.array(puck_data['Location.Y'])
            loc_z = np.array(puck_data['Location.Z'])
            velo_z_adj = velo_z + (10)*(puck_data['LocationUTC']-puck_data['LocationUTC'][start_index])
            
            # new cols we'll use to make decisions on shots 
            abs_velo = (velo_x ** 2 + velo_y ** 2 + velo_z ** 2) ** 0.5
            vel_x_fil = np.array(butter_lowpass_filter(velo_x))
            vel_y_fil = np.array(butter_lowpass_filter(velo_y))
            vel_z_fil = np.array(butter_lowpass_filter(velo_z))
            vel_z_adj_fil = np.array(butter_lowpass_filter(velo_z_adj))
            vel_fil = np.array(butter_lowpass_filter(abs_velo))
            puck_data['vel_fil'] = vel_fil
            
            # which goal line should be used, when x is positive, use the positive one (89) otherwise use the neg one (-89)
            goal_line = np.where(loc_x >= 0, NET_X_POSITION, -NET_X_POSITION)
            
            intersection_point_y = (((goal_line - loc_x) / vel_x_fil) * vel_y_fil) + loc_y
            intersection_point_y_natural = (((goal_line - loc_x) / velo_x) * velo_y) + loc_y
            intersection_point_z = (((goal_line - loc_x) / vel_x_fil) * vel_z_fil) + loc_z
            intersection_point_z_natural = (((goal_line - loc_x) / velo_x) * velo_z) + loc_z
            intersection_point_z_adj = (((goal_line - loc_x) / velo_x) * velo_z_adj) + loc_z
            
            puck_data['inter_y'] = intersection_point_y
            puck_data['inter_y_unfil'] = intersection_point_y_natural
            puck_data['inter_z'] = intersection_point_z
            puck_data['inter_z_unfil'] = intersection_point_z_natural
            puck_data['inter_z_adj'] = intersection_point_z_adj
            
            #reset index
            puck_data = puck_data.reset_index()

            # trim puck tracking data
            puck_section = puck_data[(shot_start_time + 0.05 <= puck_data.LocationUTC) & (puck_data.LocationUTC <= shot_end_time)]
    
            # return coords based on which side of the ice puck is on at the end of the shot
            outside_crease_one = puck_section.loc[(puck_section['Location.X'] >= -83) 
                                            & ((puck_section['Location.Y'] >=4) | (puck_section['Location.Y'] <=-4))]
            outside_crease_two = puck_section.loc[(puck_section['Location.X'] <= 83) 
                                            & ((puck_section['Location.Y'] >=4) | (puck_section['Location.Y'] <=-4))]
            
            #no data
            if len(puck_section) <= 1:  
                puck_x = 0
                puck_y = 0
                puck_z = 0
                        
            # jam plays within crease, no time and space to pick a corner
            elif puck_section['Location.X'].iloc[-1] < 0 and len(outside_crease_one) == 0:
                puck_x = -89
                puck_y = 0
                puck_z = 0
                
            elif puck_section['Location.X'].iloc[-1] < 0 and len(outside_crease_one) != 0:
                puck_x = -89
                puck_y = outside_crease_one.iloc[-1]['inter_y']
                puck_z = outside_crease_one.iloc[-1]['inter_z_adj']
                
            elif puck_section['Location.X'].iloc[-1] > 0 and len(outside_crease_two) == 0:
                puck_x = 89
                puck_y = 0
                puck_z = 0
            
            elif puck_section['Location.X'].iloc[-1] > 0 and len(outside_crease_two) != 0:
                puck_x = 89
                puck_y = outside_crease_two.iloc[-1]['inter_y']
                puck_z = outside_crease_two.iloc[-1]['inter_z_adj']

            # convert to Decimal
            puck_x = clean_string(str(puck_x))
            puck_y = clean_string(str(puck_y))
            puck_z = clean_string(str(puck_z))

            # output as dict
            coords_xyz = {"puck_x": puck_x, "puck_y": puck_y, "puck_z": puck_z}

            # append to dict
            shot_outcome_coords.append(coords_xyz)
    
    shot_event_df['shot_outcome_coords'] = shot_outcome_coords
    
    return shot_event_df


In [47]:
# use code above or just use csv that I saved:
shot_events_df = pd.read_csv(f'shot_events_HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125.csv')

In [32]:
event_id = 'HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125'

In [57]:
shot_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 85 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               107 non-null    int64  
 1   index                    107 non-null    int64  
 2   EventId                  107 non-null    object 
 3   ShotUTC                  107 non-null    float64
 4   Angle                    107 non-null    float64
 5   Assist1Zone              4 non-null      object 
 6   Assist2Location_X        4 non-null      object 
 7   Assist2Location_Y        4 non-null      object 
 8   Assist2Zone              4 non-null      object 
 9   AssistLocation_X         4 non-null      object 
 10  AssistLocation_Y         4 non-null      object 
 11  Ballorpuck               107 non-null    int64  
 12  BlockerLocation_X        22 non-null     object 
 13  BlockerLocation_Y        20 non-null     object 
 14  BlockerZone              2

In [38]:
shot_events_df.head(10)

,Unnamed: 0,index,EventId,ShotUTC,Angle,Assist1Zone,Assist2Location_X,Assist2Location_Y,Assist2Zone,AssistLocation_X,AssistLocation_Y,Ballorpuck,BlockerLocation_X,BlockerLocation_Y,BlockerZone,ClockMinutes,ClockSeconds,ClockTenths,ClosestDefender,ClosestDefenderDistance,Confidence,Descriptor,Distance,EndUTC,Goalie,GoalieAngle,HomeScore,IsInHomePlateArea,IsOfficial,Location_X,Location_Y,MarkerId,MarkerUTC,MinorType,NonGoalieShotBlocker,OffGoalFrameLocation_X,OffGoalFrameLocation_Y,OffGoalFrameLocation_Z,OfficialCode,OnGoalFrameArea,OnGoalFrameAreaCode,OnGoalFrameLocation_X,OnGoalFrameLocation_Y,OnGoalFrameLocation_Z,Period,PeriodNum,PlayersOnSurface_Ids,PlayersOnSurface_Teams,PuckTracking,PuckZone,RevisionNum,Shooter,ShotAssister,ShotAssister2,ShotBlockLocation_X,ShotBlockLocation_Y,ShotBlockLocation_Z,ShotCode,ShotMissed,ShotMissedCode,ShotResult,ShotType,ShotVelocity_X,ShotVelocity_Y,ShotVelocity_Z,SituationCode,SourceId,SourceNativeMarkerId,SourceType,Speed,TotalClockSec,ttl,UpdateType,VisitorScore,WasAwarded,WasBlocked,WasEmptyNet,WasGoal,WasOnTargetToGoal,WasOwnGoal,WasPenaltyShot,WasSaved,WasTipped,shot_outcome_coords
0,0,0,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637075.918,120.473,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,19,25,0,10044,7.661,100,"""HITS : E#8 19:25 - Shot. ONGOAL. V#17, Killorn.""",40,1682637076.254,10035,118.999,0,False,True,54,'-20,251,1682637079.347,EVENT_SHOT,NaN,NaN,NaN,NaN,2022030125,NaN,NaN,89,0.9021048726302325,3.016,1,1,"[{""S"":""8473986""},{""S"":""8475167""},{""S"":""8476883...","[{""S"":""VISITOR""},{""S"":""VISITOR""},{""S"":""VISITOR...","""[{""EntityId"":""1"",""OnPlayingSurface"":true,""Loc...",offense zone,9,14017,NaN,NaN,NaN,NaN,NaN,803.000,NaN,NaN,on goal,Wrist,108.36031186686796,59.12047750994818,25.908444438771234,1551,HITS,8,MA_SOURCE_TYPE_OFFICIAL_SCORING,123.439,35,1683583981.057,MA_UPDATE_TYPE_EDIT,0,False,NaN,False,False,True,False,False,True,NaN,"{'puck_x': Decimal('89'), 'puck_y': Decimal('0..."
1,1,1,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637084.382,139.297,NaN,NaN,NaN,NaN,NaN,NaN,1,'-65,23,ZONE_DEFENSE,19,18,0,14048,7.540,100,"""HITS : E#201 19:18 - Shot. VBLOCKED. H#88, Ny...",33,1682637084.993,14088,131.130,0,False,True,'-58,29,638,1682637083.347,EVENT_SHOT,14048.000,'-89.00000065042073,'-9.509935477439402,9.384,2022030125,NaN,NaN,NaN,NaN,NaN,1,1,"[{""S"":""8473986""},{""S"":""8475167""},{""S"":""8476883...","[{""S"":""VISITOR""},{""S"":""VISITOR""},{""S"":""VISITOR...","""[{""EntityId"":""1"",""OnPlayingSurface"":true,""Loc...",offense zone,16,10088,NaN,NaN,NaN,NaN,NaN,802.000,NaN,NaN,blocked,Snap,'-53.48808717992693,'-61.403450159145564,23.216853270477444,1551,HITS,201,MA_SOURCE_TYPE_OFFICIAL_SCORING,81.433,42,1683578920.176,MA_UPDATE_TYPE_EDIT,0,False,NaN,False,False,False,False,False,NaN,False,"{'puck_x': Decimal('89'), 'puck_y': Decimal('9..."
2,2,2,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637139.534,49.826,NaN,NaN,NaN,NaN,NaN,NaN,1,50,6,ZONE_DEFENSE,18,23,0,10052,13.074,100,"""HITS : E#10 18:23 - Shot. HBLOCKED. V#98, Ser...",39,1682637139.678,10035,82.913,0,False,True,34,8,1646,1682637140.347,EVENT_SHOT,10052.000,89,'-40.90091200792847,NaN,2022030125,NaN,NaN,NaN,NaN,NaN,1,1,"[{""S"":""8474564""},{""S"":""8476453""},{""S"":""8476883...","[{""S"":""VISITOR""},{""S"":""VISITOR""},{""S"":""VISITOR...","""[{""EntityId"":""1"",""OnPlayingSurface"":true,""Loc...",offense zone,13,14098,14021.000,NaN,NaN,NaN,NaN,803.000,NaN,NaN,blocked,Wrist,107.52880567641903,'-13.05301459420173,12.248511871579167,1551,HITS,10,MA_SOURCE_TYPE_OFFICIAL_SCORING,108.318,97,1683583981.057,MA_UPDATE_TYPE_EDIT,0,False,NaN,False,False,False,False,False,NaN,False,"{'puck_x': Decimal('89'), 'puck_y': Decimal('7..."
3,3,3,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637201.599,138.615,NaN,NaN,NaN,NaN,NaN,NaN,1,49,'-28,ZONE_DEFENSE,17,47,0,10052,11.147,100,"""HITS : E#11 17:47 - Shot. HBLOCKED. V#28, Cole.""",48,1682637201.695,10035,119.280,0,False,True,37,'-36,2159,1682637205.149,EVENT_SHOT,10052.000,

In [48]:
shot_events_df.iloc[0]['shot_outcome_coords']

"{'puck_x': Decimal('89'), 'puck_y': Decimal('0.5495137543841562'), 'puck_z': Decimal('4.255482757311791')}"

In [49]:
def remove_decimal(val):
    if isinstance(val, Decimal):
        return float(val)
    elif isinstance(val, dict):
        return {k: remove_decimal(v) for k, v in val.items()}
    elif isinstance(val, list):
        return [remove_decimal(v) for v in val]
    else:
        return val
    
shot_events_df['shot_outcome_coords'] = shot_events_df['shot_outcome_coords'].apply(lambda x: remove_decimal(eval(x)))

### Sniper Ability - Potential All-Star Application

In the case of players shooting on an empty net (or net with targets) we may change the code so that the calculation is being made once the puck has crossed the goal-line. 

In [50]:
def distance_to_nearest_bar(shot_events_df):
    
    distances = []
    
    for index,row in shot_events_df.iterrows():
        
        # Puck coords
        puck_x = row['shot_outcome_coords']['puck_x']
        puck_y = row['shot_outcome_coords']['puck_y']
        puck_z = row['shot_outcome_coords']['puck_z']        
    
        # Check if the puck's y coordinate is within the bounds of the goal frame
        if puck_y < -3 or puck_y > 3 or puck_z > 4:
            distances.append(np.nan)
            continue
        
        # Exclude cases where players may be purposely shooting five-hole, below blocker or glove 
        # These are strategic shot placements but we can't measure these cases
        if puck_y > -2 and puck_y < 2 and puck_z < 3:
            distances.append(np.nan)
            continue
            
        # Post
        bar1 = -3
        # Post
        bar2 = 3
        # Cross-bar
        bar3 = 4

        # Calculate the distances from the puck to each of the three bars
        dist1 = np.abs(puck_y - bar1)
        dist2 = np.abs(puck_y - bar2)
        dist3 = np.abs(puck_z - bar3)
        
        # Add to list
        distances.append(min(dist1, dist2, dist3))
        
    # Add to df
    shot_events_df['distance_to_nearest_bar'] = distances

    
    return shot_events_df

In [51]:
shot_events_df = distance_to_nearest_bar(shot_events_df)

In [52]:
shot_events_df

,Unnamed: 0,index,EventId,ShotUTC,Angle,Assist1Zone,Assist2Location_X,Assist2Location_Y,Assist2Zone,AssistLocation_X,AssistLocation_Y,Ballorpuck,BlockerLocation_X,BlockerLocation_Y,BlockerZone,ClockMinutes,ClockSeconds,ClockTenths,ClosestDefender,ClosestDefenderDistance,Confidence,Descriptor,Distance,EndUTC,Goalie,GoalieAngle,HomeScore,IsInHomePlateArea,IsOfficial,Location_X,Location_Y,MarkerId,MarkerUTC,MinorType,NonGoalieShotBlocker,OffGoalFrameLocation_X,OffGoalFrameLocation_Y,OffGoalFrameLocation_Z,OfficialCode,OnGoalFrameArea,OnGoalFrameAreaCode,OnGoalFrameLocation_X,OnGoalFrameLocation_Y,OnGoalFrameLocation_Z,Period,PeriodNum,PlayersOnSurface_Ids,PlayersOnSurface_Teams,PuckTracking,PuckZone,RevisionNum,Shooter,shot_outcome_coords,ShotAssister,ShotAssister2,ShotBlockLocation_X,ShotBlockLocation_Y,ShotBlockLocation_Z,ShotCode,ShotMissed,ShotMissedCode,ShotResult,ShotType,ShotVelocity_X,ShotVelocity_Y,ShotVelocity_Z,SituationCode,SourceId,SourceNativeMarkerId,SourceType,Speed,TotalClockSec,ttl,UpdateType,VisitorScore,WasAwarded,WasBlocked,WasEmptyNet,WasGoal,WasOnTargetToGoal,WasOwnGoal,WasPenaltyShot,WasSaved,WasTipped,distance_to_nearest_bar
0,0,0,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637075.918,120.473,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,19,25,0,10044,7.661,100,"""HITS : E#8 19:25 - Shot. ONGOAL. V#17, Killorn.""",40,1682637076.254,10035.000,118.99925709659834,0,False,True,54,'-20,251,1682637079.347,EVENT_SHOT,NaN,NaN,NaN,NaN,2022030125,NaN,NaN,89,0.9021048726302325,3.016,1,1,"[{""S"":""8473986""},{""S"":""8475167""},{""S"":""8476883...","[{""S"":""VISITOR""},{""S"":""VISITOR""},{""S"":""VISITOR...","""[{""EntityId"":""1"",""OnPlayingSurface"":true,""Loc...",offense zone,9,14017,"{'puck_x': 89.0, 'puck_y': 0.5495137543841562,...",NaN,NaN,NaN,NaN,NaN,803.000,NaN,NaN,on goal,Wrist,108.36031186686796,59.12047750994818,25.908444438771234,1551,HITS,8,MA_SOURCE_TYPE_OFFICIAL_SCORING,123.439,35,1683583981.057,MA_UPDATE_TYPE_EDIT,0,False,NaN,False,False,True,False,False,True,NaN,NaN
1,1,1,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637084.382,139.297,NaN,NaN,NaN,NaN,NaN,NaN,1,'-65,23,ZONE_DEFENSE,19,18,0,14048,7.540,100,"""HITS : E#201 19:18 - Shot. VBLOCKED. H#88, Ny...",33,1682637084.993,14088.000,131.13038500795074,0,False,True,'-58,29,638,1682637083.347,EVENT_SHOT,14048.000,'-89.00000065042073,'-9.509935477439402,9.384,2022030125,NaN,NaN,NaN,NaN,NaN,1,1,"[{""S"":""8473986""},{""S"":""8475167""},{""S"":""8476883...","[{""S"":""VISITOR""},{""S"":""VISITOR""},{""S"":""VISITOR...","""[{""EntityId"":""1"",""OnPlayingSurface"":true,""Loc...",offense zone,16,10088,"{'puck_x': 89.0, 'puck_y': 9.355491481559795, ...",NaN,NaN,NaN,NaN,NaN,802.000,NaN,NaN,blocked,Snap,'-53.48808717992693,'-61.403450159145564,23.216853270477444,1551,HITS,201,MA_SOURCE_TYPE_OFFICIAL_SCORING,81.433,42,1683578920.176,MA_UPDATE_TYPE_EDIT,0,False,NaN,False,False,False,False,False,NaN,False,NaN
2,2,2,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637139.534,49.826,NaN,NaN,NaN,NaN,NaN,NaN,1,50,6,ZONE_DEFENSE,18,23,0,10052,13.074,100,"""HITS : E#10 18:23 - Shot. HBLOCKED. V#98, Ser...",39,1682637139.678,10035.000,82.91321951682266,0,False,True,34,8,1646,1682637140.347,EVENT_SHOT,10052.000,89,'-40.90091200792847,NaN,2022030125,NaN,NaN,NaN,NaN,NaN,1,1,"[{""S"":""8474564""},{""S"":""8476453""},{""S"":""8476883...","[{""S"":""VISITOR""},{""S"":""VISITOR""},{""S"":""VISITOR...","""[{""EntityId"":""1"",""OnPlayingSurface"":true,""Loc...",offense zone,13,14098,"{'puck_x': 89.0, 'puck_y': 7.170847242896546, ...",14021.000,NaN,NaN,NaN,NaN,803.000,NaN,NaN,blocked,Wrist,107.52880567641903,'-13.05301459420173,12.248511871579167,1551,HITS,10,MA_SOURCE_TYPE_OFFICIAL_SCORING,108.318,97,1683583981.057,MA_UPDATE_TYPE_EDIT,0,False,NaN,False,False,False,False,False,NaN,False,NaN
3,3,3,HOCKEY_NHL_2023_04_27_TBL@TOR_HITS125,1682637201.599,138.615,NaN,NaN,NaN,NaN,NaN,NaN,1,49,'-28,ZONE_DEFENSE,17,47,0,10052,11.147,100,"""HITS : E#11 17:47 - Shot. HBLOCKED. V#28, Cole.""",48,1682637201

In [55]:
def calculate_sniper_ability(shot_events_df):
    
    # Group the dataframe by Shooter and calculate the mean distance to the nearest bar for each player
    avg_distance = shot_events_df.groupby("Shooter")["distance_to_nearest_bar"].mean()
    
    # Find the player with the lowest average distance to the nearest bar
    best_sniper = avg_distance.idxmin()
    
    print(f"{best_sniper} had the greatest sniper ability")
    
    # Return the dictionary of average distances
    return avg_distance.to_dict()

In [56]:
calculate_sniper_ability(shot_events_df)

14048 had the greatest sniper ability


{10002: nan,
 10003: nan,
 10012: nan,
 10016: nan,
 10019: 0.48383910050588974,
 10022: nan,
 10028: nan,
 10034: nan,
 10044: 0.425033182969881,
 10055: nan,
 10078: 0.43364385342918244,
 10088: 0.655248628187378,
 10090: nan,
 10091: nan,
 14014: nan,
 14017: nan,
 14020: nan,
 14021: nan,
 14023: nan,
 14024: nan,
 14028: nan,
 14038: 0.23632656002950458,
 14043: nan,
 14048: 0.0593070586741451,
 14071: 0.42727461418718526,
 14077: nan,
 14079: nan,
 14086: 0.40289002666513163,
 14091: 0.17690485282748347,
 14098: nan,
 22002: nan,
 22014: nan,
 22029: 0.09836758421522562,
 22037: nan,
 22073: nan,
 22091: nan,
 22097: 0.7097946991314075,
 54002: nan,
 54009: nan,
 54019: nan,
 54022: nan,
 54049: nan,
 54061: nan,
 54071: 0.5604978173849244,
 54081: nan}

In [29]:
# Additions

# 1) calculation made only when player is shooting to score .... 
    # in all star game don't need to worry about this on target shooting 
    # during game, check if shot was tipped or if there is a teammate out front
    
# 2) lift restrictions on shot outcome coords for when players are shooting on an open net (all-star game)

# 3) contest idea - sniper ability - similar to target shooting 
    # players shoot the puck 10 times
    # takes average of the calculation over the 10 shots 
    # lowest score is the ultimate sniper
    # bar down is a calculation of zero 
    
# 4) components to calc - check if posts was likely hit
# check velocity angle change at that poont to determine bar down or post in
# rely on granularity of puck between goal line and when it goes into net 